<h2 align="center">Machine Learning</h2> 
<h3 align="center">Travis Millburn<br>Fall 2024</h3> 

<center>
<img src="../images/logo.png" alt="drawing" style="width: 300px;"/>
</center>

<h3 align="center">Class 3: K Nearest Neighbors Algorithms</h3> 


# Outline 
1. Supervised vs Unsupervised models
2. K-NN 
    * K-NN Classifiers
    * Pseudocode
    * Build K-NN in SkLearn
    * Determine and critically compare effectiveness


# Until now, we've focused on data analysis ... 

# Today we build some models !

# Supervised vs Unsupervised
  
  Within the field of machine learning, there are two main types of model types: supervised learning and unsupervised learning.



<center>
<img src="../images/supervised_unsupervised_tldr.png" alt="drawing" style="width: 600px;"/>
</center>


# Supervised Learning
* Built using known values from historical data  
* Known inputs matched with known outputs  
* Generalizing from known examples:
    * The model will be able to create an output when presented with not-seen-before input in the future

Examples:
    * Identifying an email as spam or not
    * Flagging a financial transaction as fraudulent or not
    * Determining whether an elephant or monkey is present in a photo
    * Predicting a profitable financial trade
    
Two Common Types:
1. Classification  
    A) Discrete Labels
2. Regression  
    A) Continuous Labels
   

# Unsupervised Learning
* No known outputs, so we are learning about the structure of the data  
* Often used for dimensionality reduction (PCA - Principle Component Analysis)  
* Very useful for exploratory analysis, or understanding our data

# Today, we will focus on K-NN Models
K Nearest Neighbors Algorithm  

* K-NN is one of the simpler machine learning algorithms  
* We will warehouse the training dataset.
    * When we run a prediction using the model, we simply find the nearest datapoint in the training dataset  

# Let's back up.  What is classification ?
Discrete Labels:
1. Dog | Cat
2. Spam | Not Spam
3. Profitable Trade | Loser Trade
4. Apple | Orange | Pear | Banana



### Formal Classification Problem 

Mathematically, a classifier is a function or model $f$ that predicts the class label  for a given input example ${\bf x}$, that is

$$\hat{y} = f({\bf x})$$

* The value $\hat{y}$ belongs to a set $\{c_1,c_2,...,c_k\}$, each $c_i$ is a class label.


The quality of the model is inherently determined by the quality and accuracy of the training set, an important consideration upon evaluating any implementation of a model. 

Several standardized data sets that have been tested and evaluated over many years.

Scikits has a very good sample of the standardized data sets set up to be easily accessed from the standard api, as [discussed here](http://scikit-learn.org/stable/datasets/).

http://scikit-learn.org/stable/datasets/

# Classification Model using K-NN  
  
A very simple K-NN model will take only one nearest-neighbor into account.

For a new input, we find the nearest input in the training dataset, and use that output

<center>
<img src="../images/one-neighbor-knn.png" alt="drawing" style="width: 600px;"/>
 Introduction to Machine Learning; Sarah Guido, Andreas Muller
</center>



Three new inputs (stars)

# It's a Party: More Neighbors

We can build a more-sophisticated model by adding a few more neighbors to our prediction (not too many, though!)


<center>
<img src="../images/three_knn.png" alt="drawing" style="width: 600px;"/>
 Introduction to Machine Learning; Sarah Guido, Andreas Muller
</center>

The "K" in K-NN refers to an algorithm that can take an arbitrary number of neighbors.

Each neighbor gets one vote.  Most votes get the classification.

# We can display the regions of predictions

<center>
<img src="../images/knn_boundaries.JPG" alt="drawing" style="width: 1200px;"/>
</center>

# Another Example

<center>
<img src="../images/knn_example.jpg" alt="drawing" style="width: 1200px;"/>
</center>

What happens to the prediction with one neighbor instead of 5 ?

# K-NN: One Neighbor

<center>
<img src="../images/knn_1.jpg" alt="drawing" style="width: 1200px;"/>
</center>


# K-NN Five Neighbors

<center>
<img src="../images/knn_5.jpg" alt="drawing" style="width: 1200px;"/>
</center>


### Algorithm

The KNN algorithm is very simple to implement, as it does not need to be trained. The training phase merely stores the training data. For each test point, we calculate the distance of that data point to every existing data point and find the $K$ closest ones. What we return is the the most common amongst the top k classification nearest to the test point. Here's the pseudocode for _K_ Nearest Neighbors:

```
kNN:

    Learn:
        Store training set T to X_train: X_train <-- T


    Predict:
        for every point xp in X_predict:
            for every point x in X_train:
                calculate the distance d in D between x and xp
            sort D in increasing order
            take the "k" items in X_train with the smallest distances to x
            return the majority class among these k items
```

#### "Neighbors-based methods are known as non-generalizing machine learning methods, since they simply “remember” all of its training data..."

https://scikit-learn.org/stable/modules/neighbors.html

### KNN is a typical example of a lazy learner. It is called "lazy" not because of its apparent simplicity, but because it doesn't learn a discriminative function from the training data but memorizes the training dataset instead.

-Python Machine Learning - Third Edition
    Mirjalili & Raschka

### What does closest mean in this sense?  How do we calculate it?

* Distance Metric
    * Euclidean Distance
    * Manhatthan Distance
    * Minkowski with p=1 is Manhattan
    * Minkowski with p=2 is Euclidean.  
    * Minkowski is generalization of Euclidean/Manhattan distance
* sklearn allows us to use the metric parameter.

<center>
<img src="../images/euclidian_dist.jpg" alt="drawing" style="width: 900px;"/>
</center>

<center>
<img src="../images/euclidean_dist_2d.jpg" alt="drawing" style="width: 900px;"/>
</center>

### Manhattan Distance

<center>
<img src="../images/manhattan_dist_wki.jpg" alt="drawing" style="width: 900px;"/>
</center>

# K-NN Models seem straightforward.  Let's build one.

In [1]:
# let's import the census and income data we used in prior class

# First: some imports
import postgresql
import pandas
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer

with postgresql.open("pq://new_haven_ds_read:new_haven_ds_secret_99@nhds-fall-24.cwroivw0q1rc.us-east-1.rds.amazonaws.com/nhds") as db:
    ps = db.prepare('select * from nhds.uci_adults')
    res = ps()
df = pandas.DataFrame(res, columns=ps.column_names)
df.tail()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,32557
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,32558
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,32559
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,32560
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K,32561


In [3]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational_num',
       'marital_status', 'occupation', 'relationship', 'race', 'gender',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'id'],
      dtype='object')

In [4]:
df['income'].unique()

array([' <=50K', ' >50K'], dtype=object)

# See that income column at the end ?  That is the discrete variable we are estimating

We are building a model that will estimate if an adult makes over $50k..... or not.

First things first: we need to split our data into training and testing

SkLearn can do this for us!

In [6]:
# We need X and Y
x_df = df.drop(columns=['income'])    # We want everything but the response variable...in this case income
y_df = df[['income']]                 # We ONLY want reponse variable

In [9]:
# Split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df)

In [10]:
# Build a K-NN Classifier
clf = KNeighborsClassifier(n_neighbors=1)

In [11]:
# Fit to the data.......what happened !?!? 
clf.fit(x_train, y_train)

ValueError: could not convert string to float: ' Private'

In [12]:
# Let's look at that dataFrame again:
x_train.tail()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,id
8783,21,?,494638,Assoc-acdm,12,Never-married,?,Own-child,White,Male,0,0,15,United-States,8783
29069,58,Self-emp-not-inc,261230,Some-college,10,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,29070
2505,57,Self-emp-not-inc,73309,HS-grad,9,Widowed,Craft-repair,Not-in-family,White,Male,0,0,55,United-States,2506
22368,24,Private,128487,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,22369
21452,45,Self-emp-not-inc,182677,HS-grad,9,Married-spouse-absent,Craft-repair,Not-in-family,Asian-Pac-Islander,Male,0,0,40,Thailand,21453


In [13]:
df.dtypes

age                 int64
workclass          object
fnlwgt              int64
education          object
educational_num     int64
marital_status     object
occupation         object
relationship       object
race               object
gender             object
capital_gain        int64
capital_loss        int64
hours_per_week      int64
native_country     object
income             object
id                  int64
dtype: object

In [14]:
# Let's use a little Encoding magic.....we'll come back to this later.
for column in df.columns:
    if df[column].dtype == type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])

In [15]:
df.dtypes

age                int64
workclass          int32
fnlwgt             int64
education          int32
educational_num    int64
marital_status     int32
occupation         int32
relationship       int32
race               int32
gender             int32
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country     int32
income             int32
id                 int64
dtype: object

In [16]:
df.tail()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
32556,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0,32557
32557,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1,32558
32558,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0,32559
32559,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0,32560
32560,52,5,287927,11,9,2,4,5,4,0,15024,0,40,39,1,32561


In [17]:
x_df = df.drop(columns=['income'])
y_df = df['income']
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, random_state=555)
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [18]:
clf.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.70


# Wow, not bad for our first model !

In [21]:
# Do things get better or worse using more neighbors?

clf_3 = KNeighborsClassifier(n_neighbors=3)
clf_3.fit(x_train, y_train)
clf_3.predict(x_test)
print('Accuracy: {:.2f}'.format(clf_3.score(x_test, y_test)))

Accuracy: 0.75


# Things have improved!  What if we do do more?

In [22]:
# Do things get better or worse using more neighbors?

clf_10 = KNeighborsClassifier(n_neighbors=10)
clf_10.fit(x_train, y_train)
clf_10.predict(x_test)
print('Accuracy: {:.2f}'.format(clf_10.score(x_test, y_test)))

Accuracy: 0.79


In [23]:
# Remember how many columns we have:
len(df.columns) - 1  # one is response var

15

# Should we keep adding to the number of estimators?  No.

Let's try anyway ...

In [24]:
# Do things get better or worse using more neighbors?

clf_20 = KNeighborsClassifier(n_neighbors=20)
clf_20.fit(x_train, y_train)
clf_20.predict(x_test)
print('Accuracy: {:.2f}'.format(clf_20.score(x_test, y_test)))

Accuracy: 0.79


# Overfitting is always a concern

### "$K$-Nearest-Neighbor" Algorithm - dealing with overfitting

As name implies, take vote of $K$ nearest samples to address those outliers.

<center>
<img src="../images/kNN_1_3_5_9.png" width=1100/>
</center>



# For the lab, let's build another one.

# Lets go to the Titanic Dataset
# https://www.kaggle.com/c/titanic/data

# Build a K-NN model to predict survival
